In [14]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

character(0)

# 🚲 Cyclistic Bike-Share Analysis (Q1 2019 vs Q1 2020)

## 📌 Introduction

Cyclistic (formerly Divvy) is a bike-share program in Chicago that provides thousands of rides each year.  
The company’s marketing strategy depends on converting **casual riders** (single-ride or day-pass users) into **annual members**, since members generate more consistent revenue.  

The goal of this project is to analyze bike-share usage patterns between casual riders and members using Q1 data from **2019** and **2020**.  

Specifically, this analysis focuses on:
- Comparing **ride length** between casuals and members  
- Understanding **total number of rides** by rider type  
- Exploring **usage patterns by day of week and hour of day**  
- Identifying **seasonal trends (January–March)**  

The findings will help Cyclistic design marketing strategies to better target casual riders and encourage membership conversion.


In [19]:
# List all files in the Kaggle input directory
list.files("/kaggle/input/divvy-trips", recursive = TRUE)


[1] "Divvy_Trips_2019_Q1 - Divvy_Trips_2019_Q1.csv"
[2] "Divvy_Trips_2020_Q1 - Divvy_Trips_2020_Q1.csv"

In [21]:
df2019 <- read.csv("/kaggle/input/divvy-trips/Divvy_Trips_2019_Q1 - Divvy_Trips_2019_Q1.csv")
df2020 <- read.csv("/kaggle/input/divvy-trips/Divvy_Trips_2020_Q1 - Divvy_Trips_2020_Q1.csv")

head(df2019)
head(df2020)


,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,ride_length,day_of_week
,<int>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,21742443,2019-01-01 0:04:37,2019-01-01 0:11:07,2167,390,199,Wabash Ave & Grand Ave,84,Milwaukee Ave & Grand Ave,Subscriber,Male,1989,0:06:30,3
2,21742444,2019-01-01 0:08:13,2019-01-01 0:15:34,4386,441,44,State St & Randolph St,624,Dearborn St & Van Buren St (*),Subscriber,Female,1990,0:07:21,3
3,21742445,2019-01-01 0:13:23,2019-01-01 0:27:12,1524,829,15,Racine Ave & 18th St,644,Western Ave & Fillmore St (*),Subscriber,Female,1994,0:13:49,3
4,21742446,2019-01-01 0:13:45,2019-01-01 0:43:28,252,"1,783.00",123,California Ave & Milwaukee Ave,176,Clark St & Elm St,Subscriber,Male,1993,0:29:43,3
5,21742447,2019-01-01 0:14:52,2019-01-01 0:20:56,1170,364,173,Mies van der Rohe Way & Chicago Ave,35,Streeter Dr & Grand Ave,Subscriber,Male,1994,0:06:04,3
6,21742448,2019-01-01 0:15:33,2019-01-01 0:19:09,2437,216,98,LaSalle St & Washington St,49,Dearborn St & Monroe St,Subscriber,Female,1983,0:03:36,3


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>
1,EACB19130B0CDA4A,docked_bike,2020-01-21 20:06:59,2020-01-21 20:14:30,Western Ave & Leland Ave,239,Clark St & Leland Ave,326,41.9665,-87.6884,41.9671,-87.6674,member,0:07:31,3
2,8FED874C809DC021,docked_bike,2020-01-30 14:22:39,2020-01-30 14:26:22,Clark St & Montrose Ave,234,Southport Ave & Irving Park Rd,318,41.9616,-87.6660,41.9542,-87.6644,member,0:03:43,5
3,789F3C21E472CA96,docked_bike,2020-01-09 19:29:26,2020-01-09 19:32:17,Broadway & Belmont Ave,296,Wilton Ave & Belmont Ave,117,41.9401,-87.6455,41.9402,-87.6530,member,0:02:51,5
4,C9A388DAC6ABF313,docked_bike,2020-01-06 16:17:07,2020-01-06 16:25:56,Clark St & Randolph St,51,Fairbanks Ct & Grand Ave,24,41.8846,-87.6319,41.8918,-87.6206,member,0:08:49,2
5,943BC3CBECCFD662,docked_bike,2020-01-30 8:37:16,2020-01-30 8:42:48,Clinton St & Lake St,66,Wells St & Hubbard St,212,41.8856,-87.6418,41.8899,-87.6343,member,0:05:32,5
6,6D9C8A6938165C11,docked_bike,2020-01-10 12:33:05,2020-01-10 12:37:54,Wells St & Hubbard St,212,Desplaines St & Randolph St,96,41.8899,-87.6343,41.8846,-87.6446,member,0:04:49,6


### Data Cleaning in SQL
We performed the following transformations in BigQuery:
1. Converted `ride_length` from string (`HH:MM:SS`) into minutes.
2. Removed rows with negative or zero `ride_length`.
3. Created a `day_of_week` column from the ride start time.
4. Dropped duplicate rows.

Here is the query used:
```sql
CREATE OR REPLACE TABLE `cyclistic0610.cyclistic.Divvy_Trips_2019_Q1_clean` AS
WITH parsed AS (
  SELECT
    t.*,
    SAFE_CAST(REGEXP_EXTRACT(t.ride_length, r'^(\d+):') AS INT64)                AS h,
    SAFE_CAST(REGEXP_EXTRACT(t.ride_length, r'^\d+:(\d{1,2})') AS INT64)         AS m,
    SAFE_CAST(REGEXP_EXTRACT(t.ride_length, r'^\d+:\d{1,2}:(\d{1,2})') AS INT64) AS s
  FROM `cyclistic0610.cyclistic.Divvy_Trips_2019_Q1` AS t
)
SELECT
  CAST(trip_id AS STRING) AS ride_id,
  CAST(start_time AS TIMESTAMP) AS started_at,
  CAST(end_time  AS TIMESTAMP) AS ended_at,
  from_station_id  AS start_station_id,
  from_station_name,
  to_station_id    AS end_station_id,
  to_station_name,
  -- Convert old 'usertype' into modern schema
  CASE WHEN usertype = 'Subscriber' THEN 'member' ELSE 'casual' END AS member_casual,
  (COALESCE(h,0) * 60 + COALESCE(m,0) + COALESCE(s,0) / 60.0) AS ride_length_minutes,
  day_of_week
FROM parsed
WHERE (COALESCE(h,0) * 60 + COALESCE(m,0) + COALESCE(s,0) / 60.0) > 0
  AND (COALESCE(h,0) * 60 + COALESCE(m,0) + COALESCE(s,0) / 60.0) < 24*60;
  ```
And I did the same with 2020 Dataset 

```sql
CREATE OR REPLACE TABLE `cyclistic0610.cyclistic.Divvy_Trips_2020_Q1_clean` AS
WITH parsed AS (
  SELECT
    t.*,
    -- Extract hours, minutes, optional seconds from 'HH:MM[:SS]'
    SAFE_CAST(REGEXP_EXTRACT(t.ride_length, r'^(\d+):') AS INT64)                AS h,
    SAFE_CAST(REGEXP_EXTRACT(t.ride_length, r'^\d+:(\d{1,2})') AS INT64)         AS m,
    SAFE_CAST(REGEXP_EXTRACT(t.ride_length, r'^\d+:\d{1,2}:(\d{1,2})') AS INT64) AS s
  FROM `cyclistic0610.cyclistic.Divvy_Trips_2020_Q1` AS t
)
SELECT
  parsed.* EXCEPT(h, m, s),
  (COALESCE(h,0) * 60 + COALESCE(m,0) + COALESCE(s,0) / 60.0) AS ride_length_minutes
FROM parsed
WHERE (COALESCE(h,0) * 60 + COALESCE(m,0) + COALESCE(s,0) / 60.0) > 0
  AND (COALESCE(h,0) * 60 + COALESCE(m,0) + COALESCE(s,0) / 60.0) < 24*60;
```

📊 Exploratory Analysis — Q1 2019
1. Average Ride Length by Rider Type

```sql
SELECT member_casual, AVG(ride_length_minutes) AS avg_minutes
FROM `cyclistic0610.cyclistic.Divvy_Trips_2019_Q1_clean`
GROUP BY member_casual;
```
➡️ Insight:Casual riders average about 25 minutes per ride, more than double members (~12 minutes).

Suggests casuals use bikes for leisure, while members use them for quick commutes.



2. Total Rides by Rider Type

```sql
SELECT member_casual, COUNT(*) AS total_rides
FROM `cyclistic0610.cyclistic.Divvy_Trips_2019_Q1_clean`
GROUP BY member_casual;

```
➡️ Insight: Members ride much more frequently than casual riders.

Indicates strong daily commuting behavior among members.

3. Usage by Day of Week
```sql
WITH named AS (
  SELECT
    member_casual,
    day_of_week,
    CASE day_of_week
      WHEN 1 THEN 'Sun' WHEN 2 THEN 'Mon' WHEN 3 THEN 'Tue'
      WHEN 4 THEN 'Wed' WHEN 5 THEN 'Thu' WHEN 6 THEN 'Fri' WHEN 7 THEN 'Sat'
    END AS dow_name,
    ride_length_minutes
  FROM `cyclistic0610.cyclistic.Divvy_Trips_2019_Q1_clean`
)
SELECT member_casual, dow_name,
       COUNT(*) AS rides,
       AVG(ride_length_minutes) AS avg_minutes
FROM named
GROUP BY member_casual, dow_name
ORDER BY dow_name, member_casual;
```

➡️ Insight:Members dominate weekdays, especially Monday–Friday.

Casuals ride more on weekends, showing recreational behavior.

4. Hourly Ride Patterns

```sql
SELECT EXTRACT(HOUR FROM started_at) AS hour,
       member_casual,
       COUNT(*) AS rides,
       AVG(ride_length_minutes) AS avg_minutes
FROM `cyclistic0610.cyclistic.Divvy_Trips_2019_Q1_clean`
GROUP BY hour, member_casual
ORDER BY hour, member_casual;
```

➡️ Insight: Member rides peak during commute hours (8 AM & 5 PM).

Casual riders peak in the afternoon/evening, aligned with leisure activities.

5. Monthly Trend in Q1

```sql
SELECT EXTRACT(MONTH FROM started_at) AS month,
       member_casual,
       COUNT(*) AS rides,
       AVG(ride_length_minutes) AS avg_minutes
FROM `cyclistic0610.cyclistic.Divvy_Trips_2019_Q1_clean`
GROUP BY month, member_casual
ORDER BY month, member_casual;
```


➡️ Insight:Trips increase as weather improves toward March.

Casual ridership grows faster than member ridership in spring months.

6. Top 20 Start Stations

```sql
SELECT start_station_id, from_station_name AS start_station_name,
       member_casual, COUNT(*) AS rides
FROM `cyclistic0610.cyclistic.Divvy_Trips_2019_Q1_clean`
GROUP BY start_station_id, start_station_name, member_casual
ORDER BY rides DESC
LIMIT 20;
```
➡️ Insight: Casual riders use tourist-heavy stations, members use commuter hubs.


7. Duration Distribution (Median & P90)

```sql
SELECT member_casual,
       APPROX_QUANTILES(ride_length_minutes, 100)[SAFE_ORDINAL(50)] AS median_minutes,
       APPROX_QUANTILES(ride_length_minutes, 100)[SAFE_ORDINAL(90)] AS p90_minutes
FROM `cyclistic0610.cyclistic.Divvy_Trips_2019_Q1_clean`
GROUP BY member_casual;

```
➡️ Insight: Members have tighter ride distributions, casuals have long outliers.

8. Weekday vs Weekend

```sql
WITH flagged AS (
  SELECT *,
         CASE WHEN day_of_week IN (1,7) THEN 'weekend' ELSE 'weekday' END AS wk
  FROM `cyclistic0610.cyclistic.Divvy_Trips_2019_Q1_clean`
)
SELECT member_casual, wk,
       COUNT(*) AS rides,
       AVG(ride_length_minutes) AS avg_minutes
FROM flagged
GROUP BY member_casual, wk
ORDER BY wk, member_casual;

```
➡️ Insight: Casual riders dominate weekends, members dominate weekdays.

📊 Exploratory Analysis — Q1 2020
1. Average Ride Length by Rider Type

```sql
SELECT member_casual, AVG(ride_length_minutes) AS avg_minutes
FROM `cyclistic0610.cyclistic.Divvy_Trips_2020_Q1_clean`
GROUP BY member_casual;
```

➡️ Insight: Casual riders take rides about twice as long as members, consistent with 2019.
 Promote memberships by highlighting cost savings for frequent, short trips (commutes).

2. Total Rides by Rider Type

```sql
SELECT member_casual, COUNT(*) AS total_rides
FROM `cyclistic0610.cyclistic.Divvy_Trips_2020_Q1_clean`
GROUP BY member_casual;
```

➡️ Insight: Members still account for the majority of rides, showing strong loyalty.
Target casual riders with offers (e.g., first-month discount) to encourage conversion to membership.

3. Usage by Day of Week

```sql
WITH named AS (
  SELECT
    member_casual,
    day_of_week,
    CASE day_of_week
      WHEN 1 THEN 'Sun' WHEN 2 THEN 'Mon' WHEN 3 THEN 'Tue'
      WHEN 4 THEN 'Wed' WHEN 5 THEN 'Thu' WHEN 6 THEN 'Fri' WHEN 7 THEN 'Sat'
    END AS dow_name,
    ride_length_minutes
  FROM `cyclistic0610.cyclistic.Divvy_Trips_2020_Q1_clean`
)
SELECT member_casual, dow_name,
       COUNT(*) AS rides,
       AVG(ride_length_minutes) AS avg_minutes
FROM named
GROUP BY member_casual, dow_name
ORDER BY dow_name, member_casual;
```

➡️ Insight: Members dominate weekday commuting, while casuals peak on weekends.

Launch weekend promotions or leisure ride passes to capture casual demand.

4. Hourly Ride Patterns

```sql
SELECT EXTRACT(HOUR FROM started_at) AS hour,
       member_casual,
       COUNT(*) AS rides,
       AVG(ride_length_minutes) AS avg_minutes
FROM `cyclistic0610.cyclistic.Divvy_Trips_2020_Q1_clean`
GROUP BY hour, member_casual
ORDER BY hour, member_casual;
```

➡️ Insight: Members ride heavily during rush hours (8 AM & 5 PM), casuals peak in the afternoon/evening.

 Market annual memberships as the best option for commuters who ride daily at predictable times.

5. Monthly Trend in Q1

```sql
SELECT EXTRACT(MONTH FROM started_at) AS month,
       member_casual,
       COUNT(*) AS rides,
       AVG(ride_length_minutes) AS avg_minutes
FROM `cyclistic0610.cyclistic.Divvy_Trips_2020_Q1_clean`
GROUP BY month, member_casual
ORDER BY month, member_casual;
```

➡️ Insight: Both groups show growth from January → March, but casual ridership grows faster.

Use spring marketing campaigns to convert casuals while they are most active.

6. Top 20 Start Stations

```sql
SELECT start_station_id, start_station_name,
       member_casual, COUNT(*) AS rides
FROM `cyclistic0610.cyclistic.Divvy_Trips_2020_Q1_clean`
GROUP BY start_station_id, start_station_name, member_casual
ORDER BY rides DESC
LIMIT 20;
```

➡️ Insight: Casuals use stations near tourist and leisure areas, members near commuting hubs.


7. Duration Distribution (Median & P90)

```sql
SELECT member_casual,
       APPROX_QUANTILES(ride_length_minutes, 100)[SAFE_ORDINAL(50)] AS median_minutes,
       APPROX_QUANTILES(ride_length_minutes, 100)[SAFE_ORDINAL(90)] AS p90_minutes
FROM `cyclistic0610.cyclistic.Divvy_Trips_2020_Q1_clean`
GROUP BY member_casual;
```

➡️ Insight: Members have more consistent short rides; casuals have highly variable, sometimes very long rides.


8. Weekday vs Weekend

```sql
WITH flagged AS (
  SELECT *,
         CASE WHEN day_of_week IN (1,7) THEN 'weekend' ELSE 'weekday' END AS wk
  FROM `cyclistic0610.cyclistic.Divvy_Trips_2020_Q1_clean`
)
SELECT member_casual, wk,
       COUNT(*) AS rides,
       AVG(ride_length_minutes) AS avg_minutes
FROM flagged
GROUP BY member_casual, wk
ORDER BY wk, member_casual;
```

➡️ Insight: Weekends attract more casual riders; weekdays remain dominated by members.

## ✅ Conclusion & Recommendations

**Conclusion:**  
Our analysis of Cyclistic bike-share data (Q1 2019 & Q1 2020) shows clear differences between casual riders and members.  
- Members ride more often, but for shorter trips (commuting behavior).  
- Casual riders ride less often, but for much longer trips (leisure/tourism).  
- Members dominate weekday usage; casuals dominate weekends and afternoons.  
- Casual usage grows faster in spring months and focuses on downtown/tourist stations.

**Recommendations:**  
1. Target casual riders with **weekend promotions** and **tourist station advertisements**.  
2. Emphasize **membership cost savings** for frequent short trips to convert casuals into members.  
3. Launch **seasonal campaigns in spring** when casual ridership spikes.  
4. Add **member-focused benefits** for commuters, like loyalty perks for weekday usage.  

This strategy will help Cyclistic increase memberships and grow long-term revenue.
